In [2]:
import numpy as np
import sys, os, shutil
import pandas as pd
import collections as cl
import itertools as it
import timeit
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import copy as cp

#Change directory
os.chdir("C:/Users/yche465/Desktop/CS570/Project")
os.getcwd()



'C:\\Users\\yche465\\Desktop\\CS570\\Project'

### Convert raw data to PDC time series (for two-stage clustering analysis)

In [3]:
#Import raw dataset
data=pd.read_sas("prep.sas7bdat")

In [4]:
#Exclude patients that did not have 2-year follow up period
startdate=data.groupby("pid").agg('min').drop(columns=['Rx_end'])
startdate=startdate.rename(columns={'Rx_start':'Initiation'})
data2=data.merge(startdate, on="pid",how='left')
exdate=max(data['Rx_end'])-dt.timedelta(days=365*2)
data3=data2[data2['Initiation']<exdate]

In [5]:
NEWID=pd.DataFrame({"pid":np.unique(data3['pid']),"PID":np.array(range(len(np.unique(data3['pid']))))+1})
data3_=data3.merge(NEWID, on='pid', how='left')
data3=data3_.drop(columns='pid')

In [6]:
#de-identify the date
a=data3["Rx_start"] - data3["Initiation"] 
data3['Rx_start_days']=a.dt.days
a=data3["Rx_end"] - data3["Initiation"] 
data3['Rx_end_days']=a.dt.days

In [7]:
# compute length of follow up
enddate=pd.DataFrame(data3.groupby("PID").agg('max')['Rx_end']).rename(columns={'Rx_end':'LFU'})
data4=data3.merge(enddate, on="PID",how='left')
FUT=data4.groupby('PID').agg('max')[['Initiation','LFU']]
data4['obs']=np.array(range(len(data4)))
dur=FUT['LFU']-FUT['Initiation']
FUT['FUT']=dur.dt.days

In [8]:
#Remove the first observations of any users with <14 PrEP duration
length=1
while length>0:
    firstobs=data4.groupby("PID").first()
    searchnext=firstobs[np.array(firstobs['Rx_end_days']-firstobs['Rx_start_days']+1<14)]
    obs_todelete=pd.Index(np.array(searchnext['obs']))
    data4.drop(obs_todelete,inplace=True)
    length=len(obs_todelete)
    
# re-calibrate the initiation date
startdate=data4.groupby("PID").agg('min')
startdate=startdate.rename(columns={'Rx_start_days':'Initiation2_days', 'Rx_start':'Initiation2'})
startdate2=startdate[['Initiation2','Initiation2_days']]
data5=data4.merge(startdate2, on="PID",how='left')
data5['Rx_start_days2']=data5['Rx_start_days']-data5['Initiation2_days']
data5['Rx_end_days2']=data5['Rx_end_days']-data5['Initiation2_days']
#Include only the participants whose updated initiation date is earlier than the hypothetical start date of the 2-yr follow up period  
data6=data5[data5['Initiation2']<exdate]


In [9]:
#Create final data frame
NEWID=pd.DataFrame({"PID":np.unique(data6['PID']),"PID2":np.array(range(len(np.unique(data6['PID']))))+1})
data6_=data6.merge(NEWID, on='PID', how='left')
data6=data6_.drop(columns='PID')
fdata=data6[['obs','PID2','Rx_start_days2','Rx_end_days2']].rename(columns={'Rx_start_days2':'Rx_start_days','Rx_end_days2':'Rx_end_days','PID2':'PID'})

In [10]:
#Make PDC matrix (for two-stage clustering analysis)
id_pool=np.unique(fdata["PID"])
medcover=np.zeros((len(id_pool),365*2))
for k in range(len(id_pool)):
    Rx_start=np.array(fdata.loc[fdata["PID"]==id_pool[k],"Rx_start_days"])
    Rx_end=np.array(fdata.loc[fdata["PID"]==id_pool[k],"Rx_end_days"])
    
    for j in range(len(Rx_start)):
        medcover[k][Rx_start[j]:Rx_end[j]+1]=1
        
start=np.array(range(0,103*7,7))
end=start+14
PDC=np.zeros((len(id_pool),103))

for k in range(len(id_pool)):    
    for i in range(len(start)):
        PDC[k][i]=sum(medcover[k][start[i]:end[i]])/14
        

In [ ]:
#Export PDC time series data
# f= open("PDC.txt","w+")
# for i in range(len(PDC)):
#     for j in range(103):
#         if j == 102:
#             f.write("%s\n" % (PDC[i][j]))
#         else:
#             f.write("%s," % (PDC[i][j]))
# f.close()

### Create time series dataset for numbers of days under medication (for GBTM analysis)

In [12]:
#Make matrix of number of days under meds (for GBTM analysis )
start2=np.arange(0,7*105,14)
DrugDay=np.zeros((len(id_pool),len(start2)-1))

In [13]:
for k in range(len(id_pool)):    
    for i in range(1,len(start2)):
        DrugDay[k][i-1]=sum(medcover[k][start2[i-1]:start2[i]])

In [ ]:
#Export number of days under meds time series data
# f= open("DrugDay.txt","w+")
# for i in range(len(DrugDay)):
#     for j in range(52):
#         if j == 51:
#             f.write("%s\n" % (DrugDay[i][j]))
#         else:
#             f.write("%s," % (DrugDay[i][j]))
# f.close()

### Preprocessing for Stage-I Clustering:

Create a data matrix object with the following three fields: 1) the total duration spent in PrEP cessation; 2) the number of the number of PrEP interval; 3) Time of first PrEP cessation

In [15]:
#Import PDC dataset
PDC = np.loadtxt('PDC.txt', usecols=range(103), delimiter=",")

In [16]:
#1) the number of PrEP cessations (i.e., interval where PDCs were <0.57)
Cess=PDC<4/7
Cessation=[]
for k in range(len(Cess)):
    t=0
    for i in range(102):
        if Cess[k][i]==False and Cess[k][i+1]==True:
            t+=1
    Cessation.append(t)

In [17]:
START_WEEK=[]
PSTART_WEEK=[]
END_WEEK=[]
PEND_WEEK=[]
for k in range(len(Cess)):
    start_week=[]
    pstart_week=[0]
    end_week=[]
    pend_week=[]
    for i in range(1,103):
        if Cess[k][i]==True and Cess[k][i-1]==False:
            start_week.append(i)
            pend_week.append(i)
            
        elif Cess[k][i]==False and Cess[k][i-1]==True:
            end_week.append(i)
            pstart_week.append(i)
            
    START_WEEK.append(start_week)
    PSTART_WEEK.append(pstart_week)
    END_WEEK.append(end_week)
    PEND_WEEK.append(pend_week)


START_WEEK_TOT=cp.deepcopy(START_WEEK)
END_WEEK_TOT=cp.deepcopy(END_WEEK)
Pr_START_WEEK=cp.deepcopy(PSTART_WEEK)
Pr_END_WEEK=cp.deepcopy(PEND_WEEK)

for i in range(len(Cess)):
    if len(Pr_START_WEEK[i])>len(Pr_END_WEEK[i]):
        Pr_END_WEEK[i].append(103)

# make time series of start and end week
for i in range(len(Cess)):   
    if len(START_WEEK_TOT[i])>len(END_WEEK_TOT[i]):
        END_WEEK_TOT[i].append(103)
    elif len(START_WEEK_TOT[i])<len(END_WEEK_TOT[i]):
        START_WEEK_TOT[i].insert(0,0)
    elif len(START_WEEK[i])==len(END_WEEK[i]) and len(START_WEEK[i])>0 and START_WEEK[i][0]>END_WEEK[i][0]:
        START_WEEK_TOT[i].insert(0,0)
        END_WEEK_TOT[i].append(103)
    elif len(START_WEEK[i])==0 and len(START_WEEK[i])==0 and sum(Cess[i])>0:
        START_WEEK_TOT[i].insert(0,0)
        END_WEEK_TOT[i].append(103)

In [18]:
#Create list object: the total duration spent in PrEP cessation
Total_Cess_Dur=np.array([sum(Cess[i]) for i in range(len(Cess))])
#Create list object: the number of the number of PrEP interval
Num_PrEP=np.array([len(Pr_START_WEEK[i]) for i in range(len(Cess))])
#Create list object: Time of first cessation
Cess_1st=[]
for i in range(len(START_WEEK_TOT)):
    if len(START_WEEK_TOT[i])==0:
        Cess_1st.append(104)
    else:
        Cess_1st.append(START_WEEK_TOT[i][0])

In [23]:
from sklearn import preprocessing as prep
#Sample dataset
Data=np.dstack((Total_Cess_Dur,Num_PrEP,Cess_1st))[0]

In [ ]:
# import pickle
# pickle_out=open("UnSTD_DATA","wb")
# pickle.dump(Data,pickle_out)
# pickle_out.close()


### Preprocessing for Stage-II Clustering:

Create list objects for PDC time series samples (sample size: 1%, 5%, 10%, 15%) grouped by stage-I cluster memberships and a matrix object for pair-wise  DTW distance among the corresponding samples 

In [25]:
from tslearn.metrics import dtw, dtw_path
from tslearn.clustering import TimeSeriesKMeans

#Import the PDC time series data matrix grouped by stage-I cluster membership (extracted in the jupyter notebook entitled "Stage I Clustering")
import pickle
pickle_in=open("PDC_S1","rb")
loadobj=pickle.load(pickle_in)
PDC_S1=loadobj

In [6]:
#Sample size = 1%
np.random.seed(0)
samplesize=[np.int(0.01*len(PDC_S1[i])) for i in range(len(PDC_S1))]
PDC_S1_sample=[]
for i in range(len(PDC_S1)):
    selected=np.random.choice(len(PDC_S1[i]),samplesize[i])
    s= PDC_S1[i][selected]
    PDC_S1_sample.append(s)
    
DISTM=[]
for k in range(len(PDC_S1_sample)):
    num=len(PDC_S1_sample[k])
    dist_matrix=np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            dist_matrix[i,j]=dtw(PDC_S1_sample[k][i],PDC_S1_sample[k][j],global_constraint="sakoe_chiba", sakoe_chiba_radius=10)
    DISTM.append(dist_matrix)  

DISTM_1=cp.deepcopy(DISTM)
PDC_S1_sample1=cp.deepcopy(PDC_S1_sample)

In [9]:
#Sample size = 5%
np.random.seed(0)
samplesize=[np.int(0.05*len(PDC_S1[i])) for i in range(len(PDC_S1))]
PDC_S1_sample=[]
for i in range(len(PDC_S1)):
    selected=np.random.choice(len(PDC_S1[i]),samplesize[i])
    s= PDC_S1[i][selected]
    PDC_S1_sample.append(s)
    
DISTM=[]
for k in range(len(PDC_S1_sample)):
    num=len(PDC_S1_sample[k])
    dist_matrix=np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            dist_matrix[i,j]=dtw(PDC_S1_sample[k][i],PDC_S1_sample[k][j],global_constraint="sakoe_chiba", sakoe_chiba_radius=10)
    DISTM.append(dist_matrix)  

DISTM_5=cp.deepcopy(DISTM)
PDC_S1_sample5=cp.deepcopy(PDC_S1_sample)

In [10]:
#Sample size = 10%
np.random.seed(0)
samplesize=[np.int(0.10*len(PDC_S1[i])) for i in range(len(PDC_S1))]
PDC_S1_sample=[]
for i in range(len(PDC_S1)):
    selected=np.random.choice(len(PDC_S1[i]),samplesize[i])
    s= PDC_S1[i][selected]
    PDC_S1_sample.append(s)
    
DISTM=[]
for k in range(len(PDC_S1_sample)):
    num=len(PDC_S1_sample[k])
    dist_matrix=np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            dist_matrix[i,j]=dtw(PDC_S1_sample[k][i],PDC_S1_sample[k][j],global_constraint="sakoe_chiba", sakoe_chiba_radius=10)
    DISTM.append(dist_matrix)  

DISTM_10=cp.deepcopy(DISTM)
PDC_S1_sample10=cp.deepcopy(PDC_S1_sample)

In [11]:
#Sample size = 15%
np.random.seed(0)
samplesize=[np.int(0.15*len(PDC_S1[i])) for i in range(len(PDC_S1))]
PDC_S1_sample=[]
for i in range(len(PDC_S1)):
    selected=np.random.choice(len(PDC_S1[i]),samplesize[i])
    s= PDC_S1[i][selected]
    PDC_S1_sample.append(s)

DISTM=[]
for k in range(len(PDC_S1_sample)):
    num=len(PDC_S1_sample[k])
    dist_matrix=np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            dist_matrix[i,j]=dtw(PDC_S1_sample[k][i],PDC_S1_sample[k][j],global_constraint="sakoe_chiba", sakoe_chiba_radius=10)
    DISTM.append(dist_matrix)  
    
DISTM_15=cp.deepcopy(DISTM)
PDC_S1_sample15=cp.deepcopy(PDC_S1_sample)

In [12]:
store_obj={"DISTM_15":DISTM_15,"DISTM_10":DISTM_10,"DISTM_5": DISTM_5, "DISTM_1":DISTM_1,
           "PDC_S1_sample1":PDC_S1_sample1,"PDC_S1_sample5":PDC_S1_sample5,"PDC_S1_sample10":PDC_S1_sample10, "PDC_S1_sample15":PDC_S1_sample15}

In [13]:
#Export the PDC samples and precomputed distance matrix as pickle object for later use in stage-II clustering
import pickle
pickle_out=open("sPDC_DISTM","wb")
pickle.dump(store_obj,pickle_out)
pickle_out.close()
